In [13]:
import numpy as np
import itertools


# Функция для построения множества допустимых решений (МДР)
def build_admissible_solutions(edges, weights):
    """
    Строит множество допустимых решений (МДР) и таблицу значений критериев ВЦФ.

    vertices_1: Список вершин множества 1 (программы)
    vertices_2: Список вершин множества 2 (измерительные пункты)
    vertices_3: Список вершин множества 3 (спутники)
    weights: Список весов для рёбер

    return: МДР и таблица значений критериев (матрица значений ВЦФ)
    """
    # Генерация всех возможных допустимых троек
    admissible_solutions = []
    vertices_1 = list(set(edge[0] for edge in edges))
    vertices_2 = list(set(edge[1] for edge in edges))
    vertices_3 = list(set(edge[2] for edge in edges))

    for solution in itertools.combinations(edges, len(vertices_1)):
        if covers_all_vertices(solution, vertices_1, vertices_2, vertices_3):
            admissible_solutions.append(solution)

    if not admissible_solutions:
        print("Не найдено допустимых решений.")
        return [], np.array([])

    criterion_matrix = np.array(
        [calculate_criteria_values(solution, edges, weights) for solution in admissible_solutions])
    return admissible_solutions, criterion_matrix


# Проверка, покрывает ли решение все вершины
def covers_all_vertices(solution, vertices_1, vertices_2, vertices_3):
    """
    Проверяет, покрывает ли решение все вершины из каждого множества (V1, V2, V3).

    solution: Множество рёбер
    vertices_1, vertices_2, vertices_3: Множества вершин

    return: True, если решение покрывает все вершины.
    """
    v1_covered = {edge[0] for edge in solution}
    v2_covered = {edge[1] for edge in solution}
    v3_covered = {edge[2] for edge in solution}

    return v1_covered == set(vertices_1) and v2_covered == set(vertices_2) and v3_covered == set(vertices_3)


# Функция для расчета значений критериев для каждого допустимого решения
def calculate_criteria_values(solution, all_combinations, weights):
    """
    Рассчитывает значения критериев F1, F2, F3 для заданного решения.

    solution: решение в виде кортежа рёбер
    all_combinations: Все возможные комбинации рёбер
    weights: список весов для рёбер

    return: значения критериев (F1, F2, F3)
    """
    f1 = max(weights[all_combinations.index(edge)][0] for edge in solution)  # Максимальная продолжительность
    f2 = sum(weights[all_combinations.index(edge)][1] for edge in solution)  # Сумма стоимости
    f3 = np.prod([weights[all_combinations.index(edge)][2] for edge in solution])  # Произведение качества

    return f1, f2, f3


# Функция для проверки доминирования решений
def dominates(a, b, criteria_types):
    """
    Проверяет, доминирует ли решение a над решением b с учётом критериев макс/мин.

    a: np.array - первое решение
    b: np.array - второе решение
    criteria_types: Список с типом критериев ('max' или 'min')
    return: True, если a доминирует b, иначе False
    """
    is_dominating = []
    for i, (x, y) in enumerate(zip(a, b)):
        if criteria_types[i] == 'max':
            is_dominating.append(x >= y)
        else:
            is_dominating.append(x <= y)

    return all(is_dominating) and any(x != y for x, y in zip(a, b))


def find_pareto_optimal_solutions(admissible_solutions, criterion_matrix, criteria_types):
    """
    Находит Паретовское множество на основе заданных критериев.

    admissible_solutions: МДР (множество допустимых решений)
    criterion_matrix: Матрица значений критериев (ВЦФ)
    criteria_types: Список критериев (макс/мин) для каждого столбца ВЦФ

    return: Список Паретовских оптимальных решений и их критериев
    """
    # Список для хранения Паретовских решений и их критериев
    pareto_optimal_solutions = []
    pareto_criterion_matrix = []
    for i, candidate in enumerate(criterion_matrix):
        is_dominated = False
        j = 0
        while j < len(pareto_optimal_solutions):
            if dominates(pareto_criterion_matrix[j], candidate, criteria_types):
                is_dominated = True
                break
            elif dominates(candidate, pareto_criterion_matrix[j], criteria_types):
                del pareto_optimal_solutions[j]
                del pareto_criterion_matrix[j]
            else:
                j += 1

        if not is_dominated:
            pareto_optimal_solutions.append(admissible_solutions[i])
            pareto_criterion_matrix.append(candidate)

    return pareto_optimal_solutions, np.array(pareto_criterion_matrix)

    # Пример данных

edges = [("v1", "v3", "v7"),
         ("v1", "v3", "v8"),
         ("v1", "v4", "v7"),
         ("v1", "v4", "v8"),
         ("v1", "v5", "v7"),
         ("v1", "v5", "v8"),
         ("v1", "v6", "v7"),
         ("v1", "v6", "v8"),
         ("v2", "v3", "v7"),
         ("v2", "v3", "v8"),
         ("v2", "v4", "v7"),
         ("v2", "v4", "v8"),
         ("v2", "v5", "v7"),
         ("v2", "v5", "v8"),
         ("v2", "v6", "v7"),
         ("v2", "v6", "v8")]
weights = ([11, 15, 20], # Решение 1
        [12, 14, 21], # Решение 2
        [13, 13, 19], # Решение 3
        [10, 16, 18], # Решение 4
        [15, 17, 20], # Решение 5
        [14, 18, 17], # Решение 6
        [10, 10, 19], # Решение 7
        [11, 9, 20], # Решение 8
        [20, 14, 21], # Решение 9
        [18, 13, 17], # Решение 10
        [17, 8, 16], # Решение 11
        [13, 11, 10], # Решение 12
        [11, 12, 18], # Решение 13
        [19, 14, 15], # Решение 14
        [18, 15, 14], # Решение 15
        [15, 10, 14])
#    [(4, 20, 15), (7, 40, 20), (9, 50, 11), (10, 25, 16),
 #          (11, 35, 14), (5, 30, 15), (9, 50, 18), (11, 40, 9)]  # Веса для рёбер

# Построение множества допустимых решений (МДР) и таблицы критериев
admissible_solutions, criterion_matrix = build_admissible_solutions(edges, weights)

# Критерии (макс. или мин.)
criteria_types = ['max', 'min', 'max']  # Первые два критерия минимизируем, последний максимизируем

# Поиск Паретовского множества
pareto_solutions, pareto_criteria = find_pareto_optimal_solutions(admissible_solutions, criterion_matrix,
                                                                  criteria_types)

print(f"Паретовские оптимальные решения:\n{pareto_solutions}")
print(f"Критериальные значения для Паретовских решений:\n{pareto_criteria}")

Не найдено допустимых решений.
Паретовские оптимальные решения:
[]
Критериальные значения для Паретовских решений:
[]
